# Zadanie 1

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

23/03/31 09:33:37 WARN Utils: Your hostname, jakub-G3-3590 resolves to a loopback address: 127.0.1.1; using 192.168.0.171 instead (on interface wlp4s0)
23/03/31 09:33:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/31 09:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
from pyspark.sql.functions import *

filepath = "online-retail-dataset.csv"
df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load(filepath)

## NULLs
### Fill

In [8]:
df.na.fill(0).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

### Explode

In [14]:
df.withColumn("Splitted", split(col("Country"), " ")) \
  .withColumn("Exploded", explode(col("Splitted"))) \
  .show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------------+--------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|         Splitted|Exploded|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----------------+--------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|[United, Kingdom]|  United|
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|[United, Kingdom]| Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|[United, Kingdom]|  United|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|[United, Kingdom]| Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|Uni

### Drop

In [18]:
print(f"DataFrame count before: {df.count()}")
print(f"DataFrame count after : {df.na.drop().count()}")

DataFrame count before: 541909


DataFrame count after : 406829


### Regex Replace

In [35]:
df.select(col("Country"),regexp_replace(col("Country"), r"United Kingdom", "Scotland").alias("Country Replaced")).show()

+--------------+----------------+
|       Country|Country Replaced|
+--------------+----------------+
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
|United Kingdom|        Scotland|
+--------------+----------------+
only showing top 20 rows



### Regex Extract

In [44]:
df.select(regexp_extract(col("Country"), r"(United)", 1).alias("Country Extracted")).show()

+-----------------+
|Country Extracted|
+-----------------+
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
|           United|
+-----------------+
only showing top 20 rows



### If Null

In [45]:
#% sql

### Null If


In [46]:
#% sql

### Replace

In [59]:
df.na.replace(2.55, 0.).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|      0.0|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

### Array Contains

In [62]:
df.withColumn("Is UK", array_contains(split(col("Country"), " "), "United")).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|Is UK|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom| true|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| true|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom| true|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| true|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom| true|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom| true|
|   536365|    2173